<a href="https://colab.research.google.com/github/Amitgm/LLm/blob/main/Fine%20Tuning/Mistral%20Fine%20Tuning/Mistral_7B_Qlora_Fine_Tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer,DataCollatorForLanguageModeling


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [5]:
from peft import LoraConfig, PeftModel, PeftConfig, get_peft_model, prepare_model_for_kbit_training

In [6]:
from datasets import load_dataset

In [7]:
import torch

In [12]:
torch.cuda.empty_cache()

In [ ]:
model_name = "alexsherstinsky/Mistral-7B-v0.1-sharded"

# mistralai/Mistral-7B-v0.1
bnb_4bit_compute_dtype = "float16"
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"":0},quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/979 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer.max_sequence_length

AttributeError: 'LlamaTokenizerFast' object has no attribute 'max_sequence_length'

In [ ]:
dataset_name = "toughdata/quora-question-answer-dataset"

In [ ]:
dataset = load_dataset(dataset_name)

Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

In [ ]:
def create_prompt_dataset(sample):

  question = sample["question"]

  answer= sample["answer"]

  # <s>[INST] Instruction [/INST] Model answer</s>[INST] Follow-up instruction [/INST]

  prompt = f"<s>[INST] {question}  [/INST] {answer}</s>"

  tokenizer_output = tokenizer(prompt,padding=True,truncation=True)

  return tokenizer_output




In [ ]:
data = dataset["train"].map(create_prompt_dataset)

Map:   0%|          | 0/56402 [00:00<?, ? examples/s]

In [ ]:
print(data['input_ids'].shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model,lora_config)

In [ ]:
def model_trainer(data):

  training_args = TrainingArguments(
        per_device_train_batch_size = 5,
        gradient_accumulation_steps = 2,
        learning_rate = 2e-4,
        max_steps=20,
        output_dir="Experiments",
        lr_scheduler_type = "cosine",
        num_train_epochs = 1,
        logging_steps=1,
        optim = "paged_adamw_32bit",
        save_total_limit = 3,
        fp16=True,
        report_to = "tensorboard"
    )

  trainer = Trainer(
      model=model,
      train_dataset=data,
      args=training_args,
      data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False))

  model.config.use_cache= False

  trainer.train()


  return model








In [ ]:
tokenizer.eos_token_id

2

In [ ]:
model_32_bit = model_trainer(data)

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,0.785000
2,0.299400
3,0.930400
4,0.874100
5,0.773900
6,1.220600
7,1.179200
8,1.353600
9,1.383100
10,1.693800


In [ ]:
model = model_trainer(data)

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,0.876900
2,0.425700
3,1.188900
4,1.080900
5,1.003800
6,1.365200
7,1.296400
8,1.398600
9,1.509000
10,1.726400


In [ ]:
model.push_to_hub(
    "Amitgm/Mistral-7b-qlora-8-bitadam-quora-question-answer", use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Amitgm/Mistral-7b-qlora-8-bitadam-quora-question-answer/commit/65a36eac673a8b3b3f1f509f9abf7d6bd85df03e', commit_message='Upload model', commit_description='', oid='65a36eac673a8b3b3f1f509f9abf7d6bd85df03e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_32_bit.push_to_hub(
    "Amitgm/Mistral-7b-qlora-32-bitadam-quora-question-answer", use_auth_token=True
)

adapter_model.safetensors:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Amitgm/Mistral-7b-qlora-32-bitadam-quora-question-answer/commit/7f87d08888573e8790aae8e859f738bd36cbbd6d', commit_message='Upload model', commit_description='', oid='7f87d08888573e8790aae8e859f738bd36cbbd6d', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
torch.cuda.is_available()

False

In [3]:
import tensorflow as tf

# Check if TensorFlow is using GPU
gpus = tf.config.list_physical_devices('GPU')

tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:TPU_SYSTEM:0', device_type='TPU_SYSTEM'),
 PhysicalDevice(name='/physical_device:TPU:0', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:1', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:2', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:3', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:4', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:5', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:6', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:7', device_type='TPU')]

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

In [9]:
PEFT_MODEL = "Amitgm/Mistral-7b-qlora-32-bitadam-quora-question-answer"

In [10]:
base_model = "alexsherstinsky/Mistral-7B-v0.1-sharded"

In [11]:
model = AutoModelForCausalLM.from_pretrained(base_model, device_map={"":0},quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

In [28]:
def generate_answer(question):

  prompt = f"<s>[INST] {question}  [/INST] </s>"

  # prompt =  "<s>[INST] " + {question} + " [/INST] " </s>

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda")

  output = model.generate(encoding.input_ids, max_new_tokens=50)

  response = tokenizer.decode(output[0], skip_special_tokens=True)

  response_start = response.find("[/INST]")

  response_full = response[response_start+len("[/INST]"):].strip()

  return response_full

In [29]:
generate_answer("What is a proxy, and how can I use one?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'A proxy is a server that acts as an intermediary between a client and a server.  It is a server that acts as a middleman between a client and a server.  It is a server that acts as a middleman'